AlexNet算是开启了持续到现在的深度学习风潮

不过李沐老师说目前计算机视觉好像已经卷到头了？CNN可能已经被卷完了吧

整个机器学习的潮流，大概是有三个阶段
1. 神经网络
2. 核方法：无敌的SVM
3. 深度学习

核方法包括后面的随机森林啥的，有一个问题就是对特征提取很敏感，如果特征工程做的不好，那很容易就g

相较于老前辈LeNet，AlexNet在架构上其实并没有非常本质的革新，但是在以下几点做出了改进：
1. 用了dropout
2. 加了relu（支持更深的模型，同时可以有更大的梯度，减缓梯度消失）
3. 用了maxpooling（容易产生更大的梯度）
4. 改变了计算机视觉领域（感觉是整个机器学习领域）的研究范式：
    1. 以往是针对数据，进行人工特征提取，然后做SVM
    2. 现在是用各种网络（比如conv layer），自动的学习特征（当然可以加上一点基于先验知识的人工特征提取），然后扔到softmax
5. AlexNet做了一定的数据增强，不会把原始数据就扔进去

看架构就可以发现，AlexNet充满一种立大砖飞的美感......

接下来看一下代码实现，这个就不在服务器上跑了，移步colab

In [ ]:
import torch
from torch import nn
from d2l import torch as d2l

net = nn.Sequential( # 因为整体架构还算简单，所以还是用Sequential
    # 这里使用一个11*11的更大窗口来捕捉对象。
    # 同时，步幅为4，以减少输出的高度和宽度。
    # 另外，输出通道的数目远大于LeNet
    nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=1), nn.ReLU(), # 用的fashionMnist，所以通道1
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
    nn.Conv2d(96, 256, kernel_size=5, padding=2), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 使用三个连续的卷积层和较小的卷积窗口。
    # 除了最后的卷积层，输出通道的数量进一步增加。
    # 在前两个卷积层之后，汇聚层不用于减少输入的高度和宽度
    nn.Conv2d(256, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Flatten(),
    # 这里，全连接层的输出数量是LeNet中的好几倍。使用dropout层来减轻过拟合
    nn.Linear(6400, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    # 最后是输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
    nn.Linear(4096, 10))

In [ ]:
X = torch.randn(1, 1, 224, 224)
for layer in net:
    X=layer(X)
    print(layer.__class__.__name__,'output shape:\t',X.shape)

AlexNet是针对ImageNet来设计的，但是我们用的数据集是2828的，所以我们通过之前定义的函数拉大到224（没啥实际作用，只是演示

In [ ]:
batch_size = 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)

In [ ]:
lr, num_epochs = 0.01, 10
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())
# colab跑去
# colab t4 gpu跑了10分钟......

李沐老师暴论：在transformer、注意力机制之前，其实整个深度学习没有什么本质上的创新，实际上cnn或者别的网络都是类似已经经验类型的创新，调着调着发现出来一个更好的设计，这样就出来了

至于很多看起来牛逼的名词，实际上只是人为造出来的，比如relu，和各种deep